In [ ]:
import os
import csv
import numpy as np
import random
import tensorly as tl
from tensorly.decomposition import non_negative_parafac
import scipy.sparse
import sys
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

In [ ]:
def getDirectoryList(path):
    directoryList = []

    for d in os.listdir(path):
        new_path = os.path.join(path, d)
        if os.path.isfile(new_path+'/coords.csv_sparse_graph.npz_interchr.npz_tsvd.npz'):
            directoryList.append(new_path)

    return directoryList

In [ ]:
def Qmat(S,configuration,path):
    dirname = os.fsdecode(configuration)
    filename = os.path.join(path, dirname+'/coords.csv_sparse_graph.npz')
    if os.path.isfile(filename): 
        A = scipy.sparse.load_npz(filename)
        '''construct modularity matrix'''
        M = A
        k = A.sum(axis=0)
        w = A.sum(axis=None)
        M = A - np.outer(k,k)*0.5/w
        output = M.shape[0]*M.shape[1]*np.trace(np.dot(np.dot(S.transpose(),M),S))/(2.0*w) #rescale by network size
    else:
        print(filename)
        output = 0
    return output

def sample_modularity(S,cf_samples,path):
    modularity_values = []
    for configuration in cf_samples:
            modularity_values.append(Qmat(S,configuration,path))  
    return modularity_values

def random_community(Sp,configuration,path):
    np.random.shuffle(Sp)
    return Qmat(Sp,configuration,path)

def membership(factors):
    S = np.zeros(shape=factors[1][1].shape)
    for c in range(comm):
        vec = 0.5*(factors[1][1][:,c]+factors[1][2][:,c]) # take the average of the 2 factors, that should be identical
        S[:,c] = vec/tl.norm(vec,1) #normalize membership
    Sp = np.copy(S) # this copy can be used for significance testing
    return S,Sp

In [ ]:
with open('../csv/chrs.csv', newline='') as csvfile:
    chroms = list(csv.reader(csvfile,delimiter='\t'))

In [ ]:
rank = 25 # numb of components in svd. The greater this value the slower the parafac convergence 
comm = 30 # numb of communities to retrive
samples = 100 #numb of sampled 3d structured

In [ ]:
path = '/media/garner1/hdd1/gpseq/10000G'
configurations = os.listdir(path) 
configurations = getDirectoryList('/media/garner1/hdd1/gpseq/10000G')[:samples]
config_sample = random.sample(configurations, k=samples) # sample k times without replacement from configurations

In [ ]:
# T = np.zeros(shape = (samples, 3043, 3043), dtype = np.float32)
T = np.zeros(shape = (samples, 3043, 3043))
graph_idx = 0
for config in config_sample:
    dirname = os.fsdecode(config)
    filename = os.path.join(path, dirname+'/coords.csv_sparse_graph.npz_interchr.npz_tsvd.npz')
    if os.path.isfile(filename): 
        svd = np.load(filename)
        u = svd['u'][:,:rank]
        s = svd['s'][:rank]
        vt = svd['vt'][:rank,:]
        T[graph_idx,:,:] = np.dot(np.dot(u,np.diag(s)),vt)
        del svd
        continue
    else:
        T[graph_idx,:,:] = np.zeros(shape = (3043,3043),)
        continue
    graph_idx += 1
print(T.shape)   

In [ ]:
import time

start_time = time.time()

factors = non_negative_parafac(T, rank=comm, verbose=1, n_iter_max=20,tol=1e-20,init='svd')

print("--- %s seconds ---" % (time.time() - start_time))   

In [ ]:
S, Sp = membership(factors)
np.savetxt("S_graphWOintra.csv", S, delimiter=",")

In [ ]:
h1 = sample_modularity(S,config_sample,path) # S on the training data

In [ ]:
h2 = sample_modularity(S,random.sample(os.listdir(path), k=100),path) # S on the test data

In [ ]:
mu_test = np.mean(h2); sigma_test = np.std(h2)

In [ ]:
ind = 1
h3 = [random_community(Sp,config_sample[ind],path) for count in range(100)] # random S on one of the training data

In [ ]:
mu_null = np.mean(h3); sigma_null = np.std(h3)

In [ ]:
sns.set(rc={'figure.figsize':(10,10)})
labels = ['training data','test data', 'null model']
histos = [h1,h2,h3]
fig, ax = plt.subplots()
for count in range(3):
#     sns.distplot(histos[count], kde=False,norm_hist=True,label=labels[count],hist_kws=dict(alpha=0.7))
    sns.distplot(histos[count], rug=True, hist=False,label=labels[count])
    

ref = Qmat(S,config_sample[ind],path)
# ax.axvline(ref,color='red',label='z-score: '+str(np.round((ref-mu)/sigma))) # draw a red vertical line at the value of S for the example graph
plt.legend()
plt.title('HiC+GPSeq with model significance '+str(np.round((mu_test-mu_null)/(sigma_test+sigma_null))))
# print('z-score is: '+str((ref-mu)/sigma))  # z-score for a random S as the null model

In [ ]:
import pandas as pd
import plotly
import plotly.graph_objects as go

N = S.shape[0]

for i in range(comm):
    a = factors[1][1][:,i]
    b = factors[1][2][:,i]
    c = factors[1][0][:,i]
    mat = N*np.outer(S[:,i],S[:,i]) # symmetrize wrt a & b
    weigth = tl.norm(a,2)*tl.norm(b,2)*tl.norm(c,2)
    print(weigth)
    lista = [ (str(chroms[r][0])+'.'+str(chroms[r][1]),str(chroms[c][0])+'.'+str(chroms[c][1]),mat[r,c]) 
             for r in range(mat.shape[0]) for c in range(mat.shape[1]) 
             if r != c and mat[r,c] > 1.0e-2 ]
    df = pd.DataFrame(lista, columns =['bead1', 'bead2', 'Score']) 
    df.bead1 = pd.to_numeric(df.bead1, errors='coerce')
    df.bead2 = pd.to_numeric(df.bead2, errors='coerce')
    df.sort_values(['bead1','bead2'],ascending=[False, False],inplace=True)
    data = df.pivot_table(index='bead1', columns='bead2', values='Score',fill_value=0)
    
    plt.figure(figsize=(10, 10))
    fig = go.Figure(data=go.Heatmap(
                       z=data.values,
                       x=data.columns,
                       y=data.index)
                   )
    fig.update_layout(
        title='Community '+str(i)+' with weight='+str(weigth),
        xaxis = axis_template,
        yaxis = axis_template,
        showlegend = False,
        width = 1000, height = 1000,
        xaxis_title="bead#1 location on genome",
        yaxis_title="bead#2 location on genome",
    )
    axis_template = dict(range = [1,24], autorange = False,
                 showgrid = False, zeroline = False,
                 linecolor = 'black', showticklabels = True,ticks = '' )
    plotly.offline.plot(fig, filename='10000G_graphWOintra_community-'+str(i)+'.html',auto_open=False)

In [ ]:
def membership_array(path,S,comm,samples,thresh): #gives pairs property for a given community on a sampled dataset
    with open('../csv/chrs.csv', newline='') as csvfile:
        chroms = list(csv.reader(csvfile,delimiter='\t'))
    
    listPairs = []
    listaMem1=[];listaMem2=[]
    N = S.shape[0]
    for bead1 in range(N-1):
        for bead2 in range(bead1+1,N):
            m1=S[bead1,comm]
            m2=S[bead2,comm]
            if m1*m2>0 and m1>thresh and m2>thresh and chroms[bead1][0] != chroms[bead2][0]: #set threshold on membership
                listPairs.append((bead1,bead2))
                listaMem1.append(m1)
                listaMem2.append(m2)
                
    listaD=[]
    f = []
    for (dirpath, dirnames, filenames) in os.walk(path):
        for dirname in dirnames:
            if dirname.startswith('cf_'):
                f.append(os.path.join(dirpath,dirname,'coords.csv'))

#     sample_list = random.sample(f,samples)
    sample_list = f[:samples]
    count = 0
    for coordfile in sample_list:
        count+=1
        
        listd=[]
        with open(coordfile, newline='') as csvfile:
                xyz = np.asfarray(list(csv.reader(csvfile)),float)[:,:3]
        for pair in listPairs:
            bead1=pair[0]; bead2=pair[1]
            b1=xyz[bead1,:]
            b2=xyz[bead2,:]
            m1=S[bead1,comm]
            m2=S[bead2,comm]
            d = np.linalg.norm(b1-b2)
            listd.append(d)
            
        listaD.append(listd)
    return listaMem1, listaMem2, list(zip(*listaD)), listPairs

In [ ]:
import time
sns.set(rc={'figure.figsize':(10,10)})
# start = time.time()
pairslist = []
meanlist=[]
lenlist = []
for comm in range(S.shape[1]):
    list1, list2, list3, list4  = membership_array('/media/garner1/hdd1/gpseq/10000G',S,comm=comm,samples=1,thresh=1.0e-3)

    arr1=np.asarray(list1) # membership strength of the first bead
    arr2=np.asarray(list2) # membership strength of the second bead
    arr3=np.asarray(list3) # euclidean distances (pairs X structure)
    arr4=np.asarray(list4) # list of pairs
    pairslist.append(list4)
    # end = time.time()
    # print(end - start)
    print(np.mean(arr3.flatten()),len(list4))
    meanlist.append(np.mean(arr3.flatten()))
    lenlist.append(len(list4))
    sns.distplot(arr3.flatten(), rug=False,hist=False,label=str(comm))

In [ ]:
sns.scatterplot(x=lenlist, y=meanlist)

In [ ]:
'''evaluate the distribution of modularity for each community on the test data'''
path = '/media/garner1/hdd1/gpseq/10000G'
h2 = [[sample_modularity(S[:,comm],random.sample(os.listdir(path), k=10),path)] for comm in range(S.shape[1])]
# h1 = [[sample_modularity(S[:,comm],config_sample,path)] for comm in range(S.shape[1]-20)]

In [ ]:
sns.set(rc={'figure.figsize':(10,10)})
labels = [str(comm) for comm in range(S.shape[1])]
fig, ax = plt.subplots()
for count in range(S.shape[1]-20):
    sns.distplot(h2[count], rug=True, hist=False,label=labels[count])


In [ ]:
from scipy.spatial.distance import pdist, squareform

def Dmat(S,A):
    M = A
    k = A.sum(axis=0)
    w = A.sum(axis=None)
    M = A - np.outer(k,k)*0.5/w
    output = M.shape[0]*M.shape[1]*np.trace(np.dot(np.dot(S.transpose(),M),S))/(2.0*w) #rescale by networksize
    return output

def D_random_community(Sp,A):
    np.random.shuffle(Sp)
    return Dmat(Sp,A)

distance_modulairities = []
null_model = []
for coordfile in config_sample:
    with open(coordfile+'/coords.csv', newline='') as csvfile:
            xyz = np.asfarray(list(csv.reader(csvfile)),float)[:,:3]
    coordinates_array = np.array(xyz)
    dist_array = pdist(coordinates_array)
    dist_matrix = squareform(dist_array)
    dist_matrix = dist_matrix + np.eye(dist_matrix.shape[0])
    proxy_mat = 1./dist_matrix
    proxy_mat = proxy_mat - np.eye(dist_matrix.shape[0])
    distance_modulairities.append(Dmat(S,proxy_mat))
    null_model.append(D_random_community(Sp,proxy_mat))

In [ ]:
sns.set(rc={'figure.figsize':(10,10)})
lista = [distance_modulairities,null_model]
labels = [str(distro) for distro in lista]
fig, ax = plt.subplots()
count=0
# for distro in lista:
sns.distplot(distance_modulairities, rug=True, hist=False)
#     count+=1
sns.distplot(null_model, rug=True, hist=False)
